# SwinIR Denoising Project


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 🔄 Update to your actual path
%cd /content/drive/MyDrive/Courses/Ryerson/8013/swinir_denoising_colab

/content/drive/MyDrive/Courses/Ryerson/8013/swinir_denoising_colab


In [ ]:
import importlib
import models.swinir_arch
importlib.reload(models.swinir_arch)

from models.swinir_arch import SwinIR

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
!rm -r models/__pycache__ scripts/__pycache__

## Generate Noisy Test Images

In [ ]:
########### Already Generated ##################
from PIL import Image
import os
import torch
from torchvision import transforms

def add_noise_to_dataset(image_dir, output_dir, sigma, rgb=False):
    os.makedirs(output_dir, exist_ok=True)
    mode = "RGB" if rgb else "L"

    for fname in os.listdir(image_dir):
        img = Image.open(os.path.join(image_dir, fname)).convert(mode)
        img_tensor = transforms.ToTensor()(img)
        noise = torch.randn_like(img_tensor) * (sigma / 255.0)
        noisy = img_tensor + noise
        noisy_img = transforms.ToPILImage()(noisy.clamp(0., 1.))
        noisy_img.save(os.path.join(output_dir, fname))

# Grayscale
for sigma in [15, 25, 35, 50]:
    add_noise_to_dataset("datasets/BSD68/grayscale", f"datasets/BSD68/noisy_gray_sigma{sigma}", sigma, rgb=False)
    add_noise_to_dataset("datasets/Test12/images", f"datasets/Test12/noisy_gray_sigma{sigma}", sigma, rgb=False)

# RGB
for sigma in [35, 45]:
    add_noise_to_dataset("datasets/BSD68/rgb", f"datasets/BSD68/noisy_rgb_sigma{sigma}", sigma, rgb=True)

## Training

In [ ]:
from scripts.train import train_known_noise

Grayscale Known

In [ ]:
for sigma in [15, 25, 50]:
  train_known_noise("datasets/BSD400", sigma=sigma, save_path=f"checkpoints/swinir_gray_sigma{sigma}.pth", rgb=False)

Epoch 1 | Sigma: 15 | Color: Grayscale | Loss: 0.0495
Epoch 2 | Sigma: 15 | Color: Grayscale | Loss: 0.0023
Epoch 3 | Sigma: 15 | Color: Grayscale | Loss: 0.0017
Epoch 4 | Sigma: 15 | Color: Grayscale | Loss: 0.0016
Epoch 5 | Sigma: 15 | Color: Grayscale | Loss: 0.0016
Epoch 6 | Sigma: 15 | Color: Grayscale | Loss: 0.0016
Epoch 7 | Sigma: 15 | Color: Grayscale | Loss: 0.0015
Epoch 8 | Sigma: 15 | Color: Grayscale | Loss: 0.0015
Epoch 9 | Sigma: 15 | Color: Grayscale | Loss: 0.0015
Epoch 10 | Sigma: 15 | Color: Grayscale | Loss: 0.0015
Epoch 11 | Sigma: 15 | Color: Grayscale | Loss: 0.0015
Epoch 12 | Sigma: 15 | Color: Grayscale | Loss: 0.0015
Epoch 13 | Sigma: 15 | Color: Grayscale | Loss: 0.0015
Epoch 14 | Sigma: 15 | Color: Grayscale | Loss: 0.0015
Epoch 15 | Sigma: 15 | Color: Grayscale | Loss: 0.0015
Epoch 16 | Sigma: 15 | Color: Grayscale | Loss: 0.0015
Epoch 17 | Sigma: 15 | Color: Grayscale | Loss: 0.0014
Epoch 18 | Sigma: 15 | Color: Grayscale | Loss: 0.0014
Epoch 19 | Sigma: 1

RGB Known

In [ ]:
for sigma in [35, 45]:
  train_known_noise("datasets/BSD500_RGB", sigma=sigma, save_path=f"checkpoints/swinir_rgb_sigma{sigma}.pth", rgb=True)

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch 1 | Sigma: 35 | Color: RGB | Loss: 0.0126
Epoch 2 | Sigma: 35 | Color: RGB | Loss: 0.0030
Epoch 3 | Sigma: 35 | Color: RGB | Loss: 0.0029
Epoch 4 | Sigma: 35 | Color: RGB | Loss: 0.0026
Epoch 5 | Sigma: 35 | Color: RGB | Loss: 0.0026
Epoch 6 | Sigma: 35 | Color: RGB | Loss: 0.0023
Epoch 7 | Sigma: 35 | Color: RGB | Loss: 0.0022
Epoch 8 | Sigma: 35 | Color: RGB | Loss: 0.0021
Epoch 9 | Sigma: 35 | Color: RGB | Loss: 0.0020
Epoch 10 | Sigma: 35 | Color: RGB | Loss: 0.0020
Epoch 11 | Sigma: 35 | Color: RGB | Loss: 0.0019
Epoch 12 | Sigma: 35 | Color: RGB | Loss: 0.0019
Epoch 13 | Sigma: 35 | Color: RGB | Loss: 0.0019
Epoch 14 | Sigma: 35 | Color: RGB | Loss: 0.0018
Epoch 15 | Sigma: 35 | Color: RGB | Loss: 0.0018
Epoch 16 | Sigma: 35 | Color: RGB | Loss: 0.0018
Epoch 17 | Sigma: 35 | Color: RGB | Loss: 0.0018
Epoch 18 | Sigma: 35 | Color: RGB | Loss: 0.0017
Epoch 19 | Sigma: 35 | Color: RGB | Loss: 0.0017
Epoch 20 | Sigma: 35 | Color: RGB | Loss: 0.0017
Epoch 1 | Sigma: 45 | Color: 

Blind (only grayscale)

In [ ]:
from scripts.train import train_blind_noise

train_blind_noise("datasets/BSD400", save_path="checkpoints/swinir_gray_blind.pth", rgb=False)

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Blind Epoch 1 | Color: Grayscale: 100%|██████████| 16000/16000 [25:57<00:00, 10.27it/s, loss=0.00123]


Finished Epoch 1 | Avg Loss: 0.0028


Blind Epoch 2 | Color: Grayscale: 100%|██████████| 16000/16000 [25:57<00:00, 10.27it/s, loss=0.00281]


Finished Epoch 2 | Avg Loss: 0.0020


Blind Epoch 3 | Color: Grayscale: 100%|██████████| 16000/16000 [25:58<00:00, 10.26it/s, loss=0.000748]


Finished Epoch 3 | Avg Loss: 0.0019


Blind Epoch 4 | Color: Grayscale: 100%|██████████| 16000/16000 [25:56<00:00, 10.28it/s, loss=0.00121]


Finished Epoch 4 | Avg Loss: 0.0019


Blind Epoch 5 | Color: Grayscale: 100%|██████████| 16000/16000 [25:52<00:00, 10.30it/s, loss=0.000608]


Finished Epoch 5 | Avg Loss: 0.0019


## Evaluate on Test12

Known

In [ ]:
from scripts.evaluate import evaluate_model

In [ ]:
for sigma in [15, 25, 50]:
  evaluate_model(
      model_path=f"checkpoints/swinir_gray_sigma{sigma}.pth",
      clean_dir="datasets/Test12/images",
      noisy_dir=f"datasets/Test12/noisy_gray_sigma{sigma}",
      output_dir=f"results/images/gray_test12_sigma{sigma}",
      rgb=False
  )

01.png - PSNR: 28.24 dB
02.png - PSNR: 29.20 dB
03.png - PSNR: 28.21 dB
04.png - PSNR: 28.38 dB
05.png - PSNR: 28.48 dB
06.png - PSNR: 27.17 dB
07.png - PSNR: 28.26 dB
08.png - PSNR: 29.59 dB
09.png - PSNR: 27.58 dB
10.png - PSNR: 28.78 dB
11.png - PSNR: 29.27 dB
12.png - PSNR: 28.87 dB

Average PSNR across all images: 28.50 dB
01.png - PSNR: 25.69 dB
02.png - PSNR: 27.48 dB
03.png - PSNR: 26.04 dB
04.png - PSNR: 26.40 dB
05.png - PSNR: 26.17 dB
06.png - PSNR: 25.47 dB
07.png - PSNR: 25.75 dB
08.png - PSNR: 27.58 dB
09.png - PSNR: 25.05 dB
10.png - PSNR: 26.90 dB
11.png - PSNR: 27.15 dB
12.png - PSNR: 26.76 dB

Average PSNR across all images: 26.37 dB
01.png - PSNR: 24.03 dB
02.png - PSNR: 26.25 dB
03.png - PSNR: 24.16 dB
04.png - PSNR: 23.76 dB
05.png - PSNR: 24.36 dB
06.png - PSNR: 23.46 dB
07.png - PSNR: 24.04 dB
08.png - PSNR: 26.89 dB
09.png - PSNR: 23.61 dB
10.png - PSNR: 25.28 dB
11.png - PSNR: 25.70 dB
12.png - PSNR: 24.92 dB

Average PSNR across all images: 24.70 dB


Blind

In [ ]:
for sigma in [15, 25, 50]:
  evaluate_model(
    model_path="checkpoints/swinir_gray_blind.pth",
    clean_dir="datasets/Test12/images",
    noisy_dir=f"datasets/Test12/noisy_gray_sigma{sigma}",
    output_dir=f"results/images/blind_gray_test12_sigma{sigma}",
    rgb=False
  )

01.png - PSNR: 31.61 dB
02.png - PSNR: 34.56 dB
03.png - PSNR: 32.70 dB
04.png - PSNR: 31.65 dB
05.png - PSNR: 32.61 dB
06.png - PSNR: 31.13 dB
07.png - PSNR: 31.29 dB
08.png - PSNR: 34.16 dB
09.png - PSNR: 31.96 dB
10.png - PSNR: 31.98 dB
11.png - PSNR: 32.06 dB
12.png - PSNR: 31.97 dB

Average PSNR across all images: 32.31 dB
01.png - PSNR: 29.41 dB
02.png - PSNR: 32.65 dB
03.png - PSNR: 30.26 dB
04.png - PSNR: 28.80 dB
05.png - PSNR: 30.03 dB
06.png - PSNR: 28.72 dB
07.png - PSNR: 29.12 dB
08.png - PSNR: 32.09 dB
09.png - PSNR: 29.34 dB
10.png - PSNR: 29.83 dB
11.png - PSNR: 29.78 dB
12.png - PSNR: 29.61 dB

Average PSNR across all images: 29.97 dB
01.png - PSNR: 26.46 dB
02.png - PSNR: 29.46 dB
03.png - PSNR: 26.69 dB
04.png - PSNR: 25.10 dB
05.png - PSNR: 26.53 dB
06.png - PSNR: 25.48 dB
07.png - PSNR: 26.21 dB
08.png - PSNR: 28.95 dB
09.png - PSNR: 25.98 dB
10.png - PSNR: 26.76 dB
11.png - PSNR: 26.86 dB
12.png - PSNR: 26.32 dB

Average PSNR across all images: 26.73 dB


## Evaluate on BSD68

Grayscale (Known)

In [ ]:
for sigma in [15, 25, 50]:
  evaluate_model(
    model_path=f"checkpoints/swinir_gray_sigma{sigma}.pth",
    clean_dir="datasets/BSD68/grayscale",
    noisy_dir=f"datasets/BSD68/noisy_gray_sigma{sigma}",
    output_dir=f"results/images/gray_bsd68_sigma{sigma}",
    rgb=False
  )

test001.png - PSNR: 27.22 dB
test002.png - PSNR: 29.47 dB
test003.png - PSNR: 28.91 dB
test004.png - PSNR: 30.18 dB
test005.png - PSNR: 27.99 dB
test006.png - PSNR: 29.37 dB
test007.png - PSNR: 29.26 dB
test008.png - PSNR: 27.79 dB
test009.png - PSNR: 29.60 dB
test010.png - PSNR: 29.74 dB
test011.png - PSNR: 28.03 dB
test012.png - PSNR: 29.43 dB
test013.png - PSNR: 29.60 dB
test014.png - PSNR: 29.88 dB
test015.png - PSNR: 28.42 dB
test016.png - PSNR: 28.34 dB
test017.png - PSNR: 31.15 dB
test018.png - PSNR: 30.61 dB
test019.png - PSNR: 28.09 dB
test020.png - PSNR: 28.42 dB
test021.png - PSNR: 26.85 dB
test022.png - PSNR: 27.96 dB
test023.png - PSNR: 28.07 dB
test024.png - PSNR: 29.10 dB
test025.png - PSNR: 29.52 dB
test026.png - PSNR: 29.00 dB
test027.png - PSNR: 30.04 dB
test028.png - PSNR: 31.85 dB
test029.png - PSNR: 25.47 dB
test030.png - PSNR: 29.67 dB
test031.png - PSNR: 25.94 dB
test032.png - PSNR: 28.12 dB
test033.png - PSNR: 27.85 dB
test034.png - PSNR: 29.60 dB
test035.png - 

RGB

In [ ]:
for sigma in [35, 45]:
  evaluate_model(
    model_path=f"checkpoints/swinir_rgb_sigma{sigma}.pth",
    clean_dir="datasets/BSD68/rgb",
    noisy_dir=f"datasets/BSD68/noisy_rgb_sigma{sigma}",
    output_dir=f"results/images/rgb_bsd68_sigma{sigma}",
    rgb=True
  )

101085.jpg - PSNR: 24.32 dB
101087.jpg - PSNR: 26.67 dB
102061.jpg - PSNR: 26.09 dB
103070.jpg - PSNR: 26.11 dB
105025.jpg - PSNR: 25.21 dB
106024.jpg - PSNR: 27.32 dB
108005.jpg - PSNR: 25.24 dB
108070.jpg - PSNR: 24.47 dB
108082.jpg - PSNR: 25.93 dB
109053.jpg - PSNR: 25.64 dB
119082.jpg - PSNR: 25.63 dB
12084.jpg - PSNR: 24.89 dB
123074.jpg - PSNR: 25.57 dB
126007.jpg - PSNR: 26.58 dB
130026.jpg - PSNR: 24.98 dB
134035.jpg - PSNR: 25.03 dB
14037.jpg - PSNR: 28.43 dB
143090.jpg - PSNR: 28.30 dB
145086.jpg - PSNR: 25.68 dB
147091.jpg - PSNR: 26.84 dB
148026.jpg - PSNR: 24.78 dB
148089.jpg - PSNR: 24.99 dB
156065.jpg - PSNR: 24.56 dB
157055.jpg - PSNR: 25.12 dB
159008.jpg - PSNR: 26.21 dB
160068.jpg - PSNR: 25.77 dB
16077.jpg - PSNR: 25.31 dB
163085.jpg - PSNR: 25.76 dB
167062.jpg - PSNR: 30.56 dB
167083.jpg - PSNR: 23.59 dB
170057.jpg - PSNR: 25.58 dB
175032.jpg - PSNR: 23.25 dB
175043.jpg - PSNR: 24.39 dB
182053.jpg - PSNR: 25.36 dB
189080.jpg - PSNR: 26.93 dB
19021.jpg - PSNR: 25.51

Grayscale (Blind)

In [ ]:
for sigma in [15, 25, 50]:
  evaluate_model(
    model_path="checkpoints/swinir_gray_blind.pth",
    clean_dir="datasets/BSD68/grayscale",
    noisy_dir=f"datasets/BSD68/noisy_gray_sigma{sigma}",
    output_dir=f"results/images/blind_gray_bsd68_sigma{sigma}",
    rgb=False
  )

test001.png - PSNR: 27.83 dB
test002.png - PSNR: 32.37 dB
test003.png - PSNR: 32.09 dB
test004.png - PSNR: 33.00 dB
test005.png - PSNR: 29.91 dB
test006.png - PSNR: 34.93 dB
test007.png - PSNR: 31.12 dB
test008.png - PSNR: 28.87 dB
test009.png - PSNR: 31.27 dB
test010.png - PSNR: 32.00 dB
test011.png - PSNR: 31.12 dB
test012.png - PSNR: 30.82 dB
test013.png - PSNR: 31.85 dB
test014.png - PSNR: 32.97 dB
test015.png - PSNR: 29.72 dB
test016.png - PSNR: 30.60 dB
test017.png - PSNR: 34.63 dB
test018.png - PSNR: 34.94 dB
test019.png - PSNR: 29.86 dB
test020.png - PSNR: 30.94 dB
test021.png - PSNR: 29.04 dB
test022.png - PSNR: 29.47 dB
test023.png - PSNR: 31.45 dB
test024.png - PSNR: 31.00 dB
test025.png - PSNR: 31.99 dB
test026.png - PSNR: 30.77 dB
test027.png - PSNR: 32.25 dB
test028.png - PSNR: 35.06 dB
test029.png - PSNR: 26.46 dB
test030.png - PSNR: 32.14 dB
test031.png - PSNR: 27.04 dB
test032.png - PSNR: 29.05 dB
test033.png - PSNR: 30.17 dB
test034.png - PSNR: 35.04 dB
test035.png - 